In [6]:
import pandas as pd
import glob
import os

def extract_from_file_list():
    #Excel Files Paths
    file_paths = glob.glob("**/*.xlsx", recursive=True)

    #All the fileds to extract
    fields_to_extract = [
        #Input Variables
        "StdLiquidDensity (kg/m3)",
        "SulfurByWt (%)",
        "ConradsonCarbonByWt (%)",
        "NitrogenByWt (%)",
        "Distillation Mass @ X Pct (C)@ 1 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 5 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 10 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 30 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 50 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 70 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 90 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 95 (%) - TBP",
        "Distillation Mass @ X Pct (C)@ 99 (%) - TBP",

        #Output Set1
        "AromByWt (%)",
        "NaphthenesByWt (%)",
        "ParaffinsByWt (%)",

        #Output Set2
        "KinematicViscosity (cSt)@ 37.78 (C)",
        "KinematicViscosity (cSt)@ 98.89 (C)",

        #Output Set3
        "KinematicViscosity (cSt)@ 20 (C)",
        "KinematicViscosity (cSt)@ 40 (C)",
        "KinematicViscosity (cSt)@ 50 (C)",
        "KinematicViscosity (cSt)@ 100 (C)",
        "KinematicViscosity (cSt)@ 150 (C)"
    ]

    data_list = []
    print(f"Processing {len(file_paths)} specific files.")

    #Iterate through each file
    for filepath in file_paths:
        try:
            #Check if file exists
            if not os.path.exists(filepath):
                print(f"Error: File not found: {filepath}")
                continue

            #Details sheet
            try:
                df = pd.read_excel(filepath, sheet_name='Sheet1', engine='openpyxl')
            except ValueError:
                print(f"Skipping {filepath}: Sheet 'Sheet1' not found.")
                continue
            
            #Clean Column Names
            df.columns = [str(c).strip() for c in df.columns]

            #Check if required columns exist
            if 'Property' not in df.columns or 'Bulk Value' not in df.columns:
                print(f"Skipping {filepath}: Missing 'Property' or 'Bulk Value' columns.")
                continue

            #Identify crude
            crude_name = os.path.basename(filepath).replace('.xlsx', '')
            row_data = {"Crude Name": crude_name}

            #Index by Property column
            df_new = df.drop_duplicates(subset=['Property'])
            df_indexed = df_new.set_index('Property')

            #Extract fields
            for field in fields_to_extract:
                try:
                    row_data[field] = df_indexed.loc[field, 'Bulk Value']
                except KeyError:
                    row_data[field] = None

            data_list.append(row_data)

        except Exception as e:
            print(f"Error reading {filepath}: {e}")

    #Save extracted data to a single CSV file
    if data_list:
        final_df = pd.DataFrame(data_list)
        final_cols = ['Crude Name'] + fields_to_extract
        final_df = final_df[final_cols]
        
        final_df.to_csv("Extracted_CrudeData.csv", index=False)
        print("-" * 30)
        print("Done. Saved to 'Extracted_CrudeData.csv'")
        print(final_df.head())

        pd.set_option('display.max_columns', None)

    else:
        print("No valid data extracted.")

if __name__ == "__main__":
    extract_from_file_list()

Processing 53 specific files.
Skipping Basrah-Heavy-2021.xlsx: Sheet 'Sheet1' not found.
Skipping Basrah-Medium-2021.xlsx: Sheet 'Sheet1' not found.
Skipping Gippsland-2021.xlsx: Sheet 'Sheet1' not found.
Skipping Grogon-2021.xlsx: Sheet 'Sheet1' not found.
Skipping Kutubu-2021.xlsx: Sheet 'Sheet1' not found.
Skipping Shengli-1983.xlsx: Sheet 'Sheet1' not found.
------------------------------
Done. Saved to 'Extracted_CrudeData.csv'
               Crude Name  StdLiquidDensity (kg/m3)  SulfurByWt (%)  ConradsonCarbonByWt (%)  NitrogenByWt (%)  Distillation Mass @ X Pct (C)@ 1 (%) - TBP  Distillation Mass @ X Pct (C)@ 5 (%) - TBP  Distillation Mass @ X Pct (C)@ 10 (%) - TBP  Distillation Mass @ X Pct (C)@ 30 (%) - TBP  Distillation Mass @ X Pct (C)@ 50 (%) - TBP  Distillation Mass @ X Pct (C)@ 70 (%) - TBP  Distillation Mass @ X Pct (C)@ 90 (%) - TBP  Distillation Mass @ X Pct (C)@ 95 (%) - TBP  Distillation Mass @ X Pct (C)@ 99 (%) - TBP  AromByWt (%)  NaphthenesByWt (%)  ParaffinsByWt 